In [8]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchAttributeException
from selenium.webdriver.common.keys import Keys

import mechanize
import time
import datetime
import csv
import sqlite3
import re
import pandas as pd

br = mechanize.Browser()
br.set_handle_robots(False)
br.set_handle_refresh(False)
br.addheaders = [('User-agent','Chrome/52.0.2743.82')]

In [2]:
def ecare_login(user, password):
   
    browser.get('https://ecarecentral.com/Login.aspx')
    browser.find_element_by_name('txtUser').send_keys(user)
    browser.find_element_by_name('txtPassword').send_keys(password)
    time.sleep(2)
    browser.find_element_by_name('btnLogin').click()
    time.sleep(3)
    try:
        browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/button[1]/span').click()
        time.sleep(3)
    except:
        pass

def create_table(conn, id_list_csv, table_name):
    with open('humids.csv', 'rb') as f:
        reader = csv.reader(f)
        id_list = list(reader)
    cur = conn.cursor()
    cur.execute('''
    DROP TABLE IF EXISTS ?''', (table_name))
    cur.execute('''
    CREATE TABLE ? (humid TEXT, ils_id TEXT, POC_signed, April_Contact_dt DATE NULL,
    May_Contact_dt DATE NULL, June_Contact_dt DATE NULL, 
    July_Contact_dt DATE NULL, August_Contact_dt DATE NULL ''', (table_name))
    for id in id_list:
        cur.execute('''
        INSERT INTO ? (humid, ils_id, POC_signed) VALUES (?, ?, ?)
        ''',(table_name, humid[0],0,0))
    conn.commit()

def get_ils_id(humid):
    browser.get('https://ecarecentral.com/Members/MemberSearch.aspx')
    browser.find_element_by_name('QuickSearch').clear()
    time.sleep(3)
    browser.find_element_by_name('QuickSearch').send_keys(str(i) + Keys.RETURN)
    counter = 0
    while counter < 10:
            try:
                time.sleep(3)
                browser.find_element_by_id('ButtonSearch').click()
            except:
                counter += 1
                continue
            break
    ils_id = browser.find_element_by_xpath('//*[@id="tableMemberResult"]/tbody/tr/td[1]/a').get_attribute("ilsmemberid")
    return ils_id

def get_poc_signed_status():
    # 1 = signed; 2 = confirmed unsigned; 3 = more tables to check 
    browser.get('https://ecarecentral.com/Members/MemberDocuments.aspx')
    time.sleep(3)
    table_id = browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_GVItems"]/tbody')
    rows = table_id.find_elements_by_tag_name('tr')
    for row in rows:
        try:
            doc_type = row.find_elements_by_tag_name('td')[2].text
            upload_date = row.find_elements_by_tag_name('td')[4].text
            if (doc_type == 'Signed POC') & (datetime.datetime.strptime(upload_date,'%m/%d/%Y') >= datetime.datetime.strptime('2016-01-01','%Y-%m-%d')):
                return 1
            elif datetime.datetime.strptime(upload_date,'%m/%d/%Y') < datetime.datetime.strptime('2016-01-01','%Y-%m-%d'):
                return 2
        except:
            pass
    try: 
        table_2 = browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_GVItems"]/tbody/tr[17]/td/table/tbody/tr/td[2]/a')
        return 3
    except:
        return 2

def set_notes_page_size_to_50():
    #set Care plan notes page size to 50 for effency
    browser.get('https://ecarecentral.com/Members/MemberDetails.aspx?MemberID=9VhaPEZaOsc%3d')
    browser.get('https://ecarecentral.com/Members/CarePlanNotes.aspx')
    time.sleep(5)
    browser.find_element_by_xpath('//*[@id="ddlSortBy"]/option[3]').click()
    time.sleep(5)
    browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_CarePlanNotes_ddlPageSize"]/option[10]').click()
    time.sleep(5)
    
def scrape_CarePlanNotes_page(contact_details):
    table_id = browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_CarePlanNotes_wdgNotes"]/table/tbody/tr[1]/td/table/tbody[2]/tr/td/div[2]/table/tbody')
    rows = table_id.find_elements_by_tag_name('tr')
    for row in rows:
        try:
            log_text = row.find_elements_by_tag_name('td')[0].find_element_by_tag_name('span').text

            event_date_regex = '(?:Event Date: )(.*?)(?: -)'
            event_date = re.findall(event_date_regex, log_text)[0]
            contact_method_regex = '(?:Event Date: )(?:.*?)(?: - )(.*?)(?: -)'
            contact_method = re.findall(contact_method_regex, log_text)[0]
            contact_reason_regex = '(?:Event Date: )(?:.*?)(?: - )(?:.*?)(?: -)(.*?)(?: -)'
            contact_reason = re.findall(contact_reason_regex, log_text)[0]
        except:
            continue
        contact_details.append([event_date, contact_method, contact_reason]) 
    
def get_monthly_contact_dates():
    browser.get('https://ecarecentral.com/Members/CarePlanNotes.aspx')
    
    contact_details = []
    try: #addesses case where no care plan exists for member
        scrape_CarePlanNotes_page(contact_details)
    except:
        months_dict = {}
        return months_dict
    
    #try:
        #links_bar = browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_CarePlanNotes_wdgNotes"]/table/tbody/tr[3]/td/div')
        #links = links_bar.find_elements_by_tag_name('span')
        #for link in links[1:]:
            #link.click()
            #time.sleep(7)
            #scrape_CarePlanNotes_page(contact_details)
    #except:
        #pass

    months_dict = {}
    for i in contact_details:
        if i[2] == ' Monthly Contact':
            date = datetime.datetime.strptime(i[0],'%m/%d/%Y').date()
            if date.month == 4:
                months_dict['April'] = date
            if date.month == 5:
                months_dict['May'] = date
            if date.month == 6:
                months_dict['June'] = date
            if date.month == 7:
                months_dict['July'] = date
            if date.month == 8:
                months_dict['August'] = date
    return months_dict
    

In [12]:
conn = sqlite3.connect('Ecare_data.sqlite')                       
cur = conn.cursor()
user = "xxx@xxx.com"
password = 'xxxxx'
table_name = 'Ecare'
id_list_csv = 'humids.csv'

In [4]:
#conn = sqlite3.connect('Ecare_data.sqlite')
#id_list_csv = 'humids.csv'
#table_name = 'Ecare'
#create_table(conn, id_list_csv, table_name)

get ils_member_id for each humid

In [5]:
#browser = webdriver.Chrome(executable_path='C:\Users\cxc1017\chromedriver.exe')
#ecare_login(user, password)

sqlstr = '''
SELECT humid 
FROM Ecare
WHERE ils_id = 0
'''
humid_list = []
for row in cur.execute(sqlstr):
    humid_list.append(row[0])

for humid in humid_list:
    ils_id = get_ils_id(humid)
    
    cur.execute('''
    UPDATE Ecare
    SET ils_id = ?
    WHERE humid = ?
    ''',(ils_id,humid))
    conn.commit()

Update POC signed status and monthly contact dates

In [4]:
sqlstr = '''
SELECT ils_id
FROM Ecare
WHERE POC_signed = 0
'''
ils_id_list = []
for row in cur.execute(sqlstr):
    ils_id_list.append(row[0])

browser = webdriver.Chrome(executable_path='C:\Users\cxc1017\chromedriver.exe')
ecare_login(user, password)

set_notes_page_size_to_50()

for ils_id in ils_id_list:
    counter = 0
    while counter < 5:
        try:
            member_url = 'https://ecarecentral.com/Members/MemberDetails.aspx?MemberID=%s' % (ils_id)
            browser.get(member_url)
            is_poc_signed = get_poc_signed_status()
            months_dict = get_monthly_contact_dates()
            April = datetime.date(2099, 12, 31)
            try:
                April = months_dict['April']
            except:
                pass
            May = datetime.date(2099, 12, 31)
            try:
                May = months_dict['May']
            except:
                pass
            June = datetime.date(2099, 12, 31)
            try:
                June = months_dict['June']
            except:
                pass
            July = datetime.date(2099, 12, 31)
            try:
                July = months_dict['July']
            except:
                pass
            August = datetime.date(2099, 12, 31)
            try:
                August = months_dict['August']
            except:
                pass
            cur.execute('''
            UPDATE Ecare
            SET POC_signed = ?,
            April_Contact_dt = ?,
            May_Contact_dt = ?,
            June_Contact_dt = ?,
            July_Contact_dt = ?,
            August_Contact_dt = ?
            WHERE ils_id = ?
            ''',(is_poc_signed, April, May, June, July, August, ils_id))
            conn.commit()
            print 'ils: %s updated' % (ils_id)
            break
        except:
            counter += 1
            time.sleep(10)
            continue
    


ils: 7Hz7RIbjGoI%3d updated
ils: 2f6fSA28O58%3d updated
ils: MvHhomoBol0%3d updated
ils: UggTyHcShdo%3d updated
ils: 0h9qpvohGCk%3d updated
ils: EPfxdDUe44I%3d updated
ils: yWvYnbvSH7k%3d updated
ils: wG17BuJPuCI%3d updated
ils: TAznO6YXk88%3d updated
ils: w5fdhi7i4tY%3d updated
ils: 6IpZ5kjj4Ow%3d updated
ils: mB6YKhYDJSs%3d updated
ils: %2fEkG0bcUsX4%3d updated
ils: iGI4mkk00FI%3d updated
ils: EV3Mm8YwSDg%3d updated
ils: QqIw2jd2%2bxM%3d updated
ils: mbEZ4MTgkGc%3d updated
ils: wfEX7pxshUo%3d updated
ils: D1rfHEj8H98%3d updated
ils: HbmHbp3sLkI%3d updated
ils: U9GgC3LMPNo%3d updated
ils: OMAVcMjnCRk%3d updated
ils: DRLi4yuzECQ%3d updated
ils: LLHRBWny8fQ%3d updated
ils: sbp2QNWw0os%3d updated
ils: NCL6uGI1a98%3d updated
ils: L55agpNK2wo%3d updated
ils: WHK%2fcz7hAGQ%3d updated
ils: YKyg8J1qULg%3d updated
ils: bxMLvOQXVs0%3d updated
ils: bFDDeliCyE0%3d updated
ils: pvRkjvTn6eM%3d updated
ils: Lbt%2bWsKVjDk%3d updated
ils: j1BC%2fHm1YDM%3d updated
ils: jWfxX2sxvRE%3d updated
ils: qKVTq

In [13]:
f = open('ecare_output.csv', 'w')
cur.execute('select * from ecare')
df = pd.DataFrame(cur.fetchall())
df.to_csv(f, header=True)

f.close()
cur.close()
conn.close()